# 近似動的計画法

参考
* [RL Theory Lecture 8](https://rltheory.github.io/w2022-lecture-notes/planning-in-mdps/lec8/)

強化学習アルゴリズムは動的計画法にさまざまな近似（探索や関数近似など）を盛り込んだフレームワークと言えます。
よって、近似誤差が動的計画法に与える影響を解析することで、強化学習の近似誤差に対する脆弱性を見積もることができそうです。
このような解析手段は**近似動的計画法**として知られています。

## 近似方策反復法

**参考**
* [RL Theory Lecture 8](https://tadashik.github.io/rltheory-jp/lecture-notes/planning-in-mdps/lec8/)
* [Leverage the Average: an Analysis of KL Regularization in Reinforcement Learning](https://arxiv.org/abs/2003.14089)：表記を参考にしました

**表記**

* $q$についての貪欲方策：$\mathcal{G}(q) = \argmax_{\pi} \langle q, \pi \rangle$
* 方策による状態遷移：$P_\pi q = P \langle q, \pi \rangle$
* ベルマン期待作用素：$T_\pi q = r + \gamma P \langle q, \pi \rangle$
* ベルマン最適作用素：$T q = r + \gamma P \max_{\pi} \langle q, \pi \rangle$
* 報酬関数の範囲：$r \in [0, 1]^{S\times A}$

近似方策反復法は近似誤差が**方策反復法**の漸近的な挙動に与える影響を解析するためのフレームワークです。
近似方策反復法では次の更新を繰り返します。

$$
\begin{cases}
\pi_{k+1} \in \mathcal{G}(q_k) \\
q_{k+1} = (T_{\pi_{k+1}})^\infty q_k + \varepsilon_{k+1} = q_{\pi_{k+1}} + \varepsilon_{k+1}
\end{cases}
$$

ここで、$\varepsilon_k$はなんらかの近似誤差（例えばNNなどの近似誤差）を表しています。

### 近似方策反復法の誤差伝搬解析

近似方策反復法の$k$ステップ目の性能$\|q_* - q_{\pi_{k}}\|_\infty$は次のように誤差に影響されます。

$$\left\|q_*-q_{\pi_k}\right\|_{\infty} \leq \frac{\gamma^k}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq k-1}\left\|\varepsilon_s\right\|_{\infty}$$

これを示すために、次の**価値差分方程式**と**指数的改善定理**が必要になります。

---

**補題１：価値差分方程式**

$q_{\pi^{\prime}}=\left(I-\gamma P_{\pi^{\prime}}\right)^{-1} r$であるから、

$$\begin{aligned} q_{\pi^{\prime}}-q_\pi & =\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[r-\left(I-\gamma P_{\pi^{\prime}}\right) q_\pi\right] \\ & =\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[T_{\pi^{\prime}} q_\pi-q_\pi\right] .\end{aligned}$$

---

---

**補題２：指数的改善定理**

$\pi^\prime$を任意の方策として、$\varepsilon$を次で定義します。

$$T q_\pi=T_{\pi^{\prime}} q_\pi+\varepsilon$$

このとき、

$$
\left\|q_*-q_{\pi^{\prime}}\right\|_{\infty} \leq \gamma\left\|q_*-q_\pi\right\|_{\infty}+\frac{1}{1-\gamma}\|\varepsilon\|_{\infty}
$$

が成立します。証明してみましょう。　

まず、$T_{\pi_*} q_*=q_*$です。よって、

$$
\begin{aligned}
q_*-q_{\pi^{\prime}} & =T_{\pi_*} q_*-T_{\pi_*} q_\pi+\overbrace{T_{\pi_*} q_\pi}^{\leq T q_\pi}-T_{\pi^{\prime}} q_\pi+T_{\pi^{\prime}} q_\pi-T_{\pi^{\prime}} q_{\pi^{\prime}} \\
& \leq \gamma P_{\pi_*}\left(q_*-q_\pi\right)+\varepsilon+\gamma P_{\pi^{\prime}}\left(q_\pi-q_{\pi^{\prime}}\right)
\end{aligned}
$$

が成り立ちます。ここで、$T q_\pi=T_{\pi^{\prime}} q_\pi+\varepsilon$の定義を利用しています。

また、$q_\pi=T_\pi q^\pi \leq T q^\pi$と価値差分方程式を利用すると、

$$
q_\pi-q_{\pi^{\prime}}=\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[q_\pi-T_{\pi^{\prime}} q_\pi\right] \leq\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[T q_\pi-\left(T q_\pi-\varepsilon\right)\right]=\left(I-\gamma P_{\pi^{\prime}}\right)^{-1} \varepsilon
$$

が成立します。ここまでに求めた不等式二つを合体させて、

$$q_*-q_{\pi^{\prime}} \leq \gamma P_{\pi_*}\left(q_*-q_\pi\right)+\left(I-\gamma P_{\pi^{\prime}}\right)^{-1} \varepsilon$$

が得られます。あとは両辺で最大値ノルムをとり、三角不等式と$\left\|\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\right\|_{\infty} \leq 1 /(1-\gamma)$を使えば簡単です。

---

**証明：誤差伝搬解析**

この指数的改善定理を使うと、次のように帰納法で誤差バウンドを示すことができます。

まず、$k=1$の時を考えましょう。指数的改善定理から、　

$$
\left\|q_*-q_{\pi_1}\right\| \leq \gamma\left\|q_*-q_{\pi_0}\right\|+\frac{1}{1-\gamma}\left\|\varepsilon_0\right\|_{\infty}
$$

が成り立ちます。報酬の最大値が１なので、$\|q_* - q_{\pi_0}\| \leq 1 / (1 - \gamma)$です。よって、

$$
\left\|q_*-q_{\pi_1}\right\| \leq \gamma\left\|q_*-q_{\pi_0}\right\|+\frac{1}{1-\gamma}\left\|\varepsilon_0\right\|_{\infty} \leq \frac{\gamma^1}{1-\gamma}+\frac{1}{(1-\gamma)^2}\left\|\varepsilon_0\right\|_{\infty}
$$

です。
つづいて、近似方策反復法の$k=i (i>1)$ステップ目の性能$\|q_* - q_{\pi_{k}}\|_\infty$が次のようにバウンドされていると仮定します。

$$\left\|q_*-q_{\pi_i}\right\|_{\infty} \leq \frac{\gamma^i}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}$$

このとき、

$$
\begin{aligned}
\left\|q_*-q_{i+1}\right\| & \leq \gamma\left\|q_*-q_{\pi_i}\right\|_{\infty}+\frac{1}{1-\gamma}\left\|\varepsilon_i\right\|_{\infty}\text { (指数的改善定理) } \\
& \leq \gamma\left(\frac{\gamma^i}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}\right)+\frac{1}{1-\gamma}\left\|\varepsilon_i\right\|_{\infty} \quad \text { (帰納法の仮定) } \\
& =\frac{\gamma^{i+1}}{1-\gamma}+\frac{\gamma}{(1-\gamma)^2} \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}+\frac{1}{1-\gamma}\left\|\varepsilon_i\right\|_{\infty} \\
& =\frac{\gamma^{i+1}}{1-\gamma}+\frac{1}{(1-\gamma)^2}\left(\gamma \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}+(1-\gamma)\left\|\varepsilon_i\right\|_{\infty}\right) \\
& \leq \frac{\gamma^{i+1}}{1-\gamma}+\frac{1}{(1-\gamma)^2}\left(\gamma \max _{0 \leq s \leq i}\left\|\varepsilon_s\right\|_{\infty}+(1-\gamma) \max _{0 \leq s \leq i}\left\|\varepsilon_s\right\|_{\infty}\right) \\
& =\frac{\gamma^{i+1}}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq i}\left\|\varepsilon_s\right\|_{\infty}
\end{aligned}
$$

以上で証明終わりです。

---

## 近似方策反復法 + KL正則化 + エントロピー正則化

**参考**

* [Leverage the Average: an Analysis of KL Regularization in Reinforcement Learning](https://arxiv.org/abs/2003.14089)：表記を参考にしました

**表記**

* 正則化された貪欲方策：$\mathcal{G}_\mu^{\lambda, \tau}(q)=\operatorname{argmax}_{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}(\langle\pi, q\rangle-\lambda \mathrm{KL}(\pi \| \mu)+\tau \mathcal{H}(\pi))$
* 正則化されたベルマン期待作用素：$T_{\pi \mid \mu}^{\lambda, \tau} q=r+\gamma P(\langle\pi, q\rangle-\lambda \operatorname{KL}(\pi \| \mu)+\tau \mathcal{H}(\pi))$
* 正則化されたベルマン最適作用素：$T_{\mu}^{\lambda, \tau} q=T_{\mathcal{G}_\mu^{\lambda, \tau}(q) \mid \mu}^{\lambda, \tau} q$
* $T_{\pi\mid \mu}^{\lambda, \tau}$の不動点：$q_{\pi\mid \mu}^{\lambda, \tau}$（エントロピーのみの場合は$\lambda$と$\mu$は省略します）
* $T_{\mu}^{\lambda, \tau}$の不動点：$q_{*\mid \mu}^{\lambda, \tau}=\max _\pi q_{\pi\mid \mu}^{\lambda, \tau}$
* 正則化された最適方策：$\pi_{*\mid \mu}^{\lambda, \tau}=\mathcal{G}_\mu^{\lambda, \tau}\left(q_{*\mid \mu}^{\lambda, \tau}\right)$
* 誤差の累積和：$E_k^\beta = (1-\beta) \sum_{j=1}^k \beta^{k-j} \epsilon_j$
* $v_{\max }^\tau=\frac{1+\tau \ln |\mathcal{A}|}{1-\gamma}$


通常の近似価値反復法では性能のバウンドが「過去の誤差の最大値」に依存していました。
これは**方策反復法が誤差に非常に弱い**ことを指し示しています（Q学習やDQNも同様です）。

一方で、次のように**KL正則化された方策反復法**を考えてみましょう。
$$
\begin{cases}
\pi_{k+1} \in \mathcal{G}_{\pi_k}^{\lambda, \tau}(q_k) \\
q_{k+1} = (T_{\pi_{k+1}\mid \pi_k}^{\lambda, \tau})^\infty q_k + \varepsilon_{k+1} = q_{\pi_{k+1}\mid \pi_k}^{\lambda, \tau} + \varepsilon_{k+1}
\end{cases}
$$

このような方策更新側の正則化は誤差に対する頑健性を著しく上げることが知られています。見てみましょう。

### 方策更新のルジャンドル変換

正則化された貪欲方策を改めてよく見てみます

$$
\mathcal{G}_\mu^{\lambda, \tau}(q)=\operatorname{argmax}_{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}(\langle\pi, q\rangle-\lambda \mathrm{KL}(\pi \| \mu)+\tau \mathcal{H}(\pi))
$$

右辺を変形してみましょう。

$$
\begin{aligned}
\langle\pi, q\rangle-\lambda \mathrm{KL}(\pi \| \mu)+\tau \mathcal{H}(\pi) & =\langle\pi, q\rangle-\lambda\langle\pi, \ln \pi-\ln \mu\rangle-\tau\langle\pi, \ln \pi\rangle \\
& =\langle\pi, q+\lambda \ln \mu\rangle-(\lambda+\tau)\langle\pi, \ln \pi\rangle \\
& =(\lambda+\tau)\left(\left\langle\pi, \frac{q+\lambda \ln \mu}{\lambda+\tau}\right\rangle+\mathcal{H}(\pi)\right) .
\end{aligned}
$$

ここで、エントロピーについての[ルジャンドル変換](CVX_convex_functions.ipynb)の解析解より、

* 最大値：$\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}(\langle\pi, q\rangle+\mathcal{H}(\pi)) =\ln \langle\mathbf{1}, \exp q\rangle \in \mathbb{R}^{\mathcal{S}}$
* 最大点：$\underset{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}{\operatorname{argmax}}(\langle\pi, q\rangle+\mathcal{H}(\pi)) =\frac{\exp q}{\langle\mathbf{1}, \exp q\rangle} \in \mathbb{R}^{\mathcal{S} \times \mathcal{A}}$

が求まるので、右辺の解析解は

* 最大値：$\max _{\pi \in \Delta_{\mathcal{A}}}(\langle\pi, q\rangle-\lambda \mathrm{KL}(\pi \| \mu)+\tau \mathcal{H}(\pi))=\frac{1}{\alpha} \ln \left\langle\mu^{\beta}, \exp (\alpha q) \right\rangle$
* 最大点：$\underset{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}{\operatorname{argmax}}(\langle\pi, q\rangle-\lambda \mathrm{KL}(\pi \| \mu)+\tau \mathcal{H}(\pi))\propto {\mu^{\beta} \exp (\alpha q) }$

になります。
ここで、$\alpha=1 / (\lambda + \tau)$, $\beta=\lambda / (\lambda + \tau)$としました。

これを使って$\pi_{k+1}$を書き下してみましょう。

$$
\begin{aligned}
\pi_{k+1} &\propto \pi_k^\beta \exp (\alpha q_k) \\
&\propto \pi_{k-1}^{\beta^2} \exp (\alpha \beta q_{k-1} + \alpha q_k)\\
&\propto \pi_{k-2}^{\beta^3} \exp (\alpha \beta^2 q_{k-2} + \alpha \beta q_{k-1} + \alpha q_k)\\
& \cdots \\
&\propto \exp \left(\alpha \sum_{j=0}^{k}\beta^{k-j} q_{j}\right)
= \exp \left(\frac{1-\beta}{\tau} \sum_{j=0}^{k}\beta^{k-j} q_{j}\right)
\end{aligned}
$$

ここで$h_{k+1}=\beta h_k+ (1 - \beta) q_{k+1} = (1 - \beta) \sum_{j=0}^{k}\beta^{k-j} q_{j}$を導入してみます。
これを使うと、正則化された方策反復法は次のように書き換えることができます。

$$
\begin{cases}
\pi_{k+1} &\in \mathcal{G}_{\pi_k}^{0, \tau}(h_k) \\
q_{k+1} &= q_{\pi_{k+1}\mid \pi_k}^{\lambda, \tau} + \varepsilon_{k+1}\\
h_{k+1}&=\beta h_k+(1-\beta) q_{k+1} \text { with } \beta=\frac{\lambda}{\lambda+\tau}
\end{cases}
$$



---

**正則化の直感的な効果**

正則化の導入前と導入後を比較してみましょう（KLのみの式の導出は省略します。論文参照。）：

|       正則化       |  方策の更新式 |
|:------------:|:-:|
|  正則化なし  | $\operatorname{argmax}_{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}\langle\pi, q_k\rangle$  |
|      KL      | $\exp \left(\frac{1}{\lambda}\sum_{j=0}^k q_j\right)$ |
|      KL + エントロピー   | $\exp \left(\alpha \sum_{j=0}^{k}\beta^{k-j} q_{j}\right)$ |

この表から、正則化がない場合は「直前のQ値」だけを見ており、正則化がある場合は「過去のQ値の和」を見ていることがわかります。さらに、KLのみよりもKL+エントロピーの方が「$\beta$によって過去の情報を忘れやすい」ことがわかります。

ここから何が言えるでしょうか？例えば直前のQ値にとても大きい誤差が乗った状況を考えてみましょう。

1. 直前のQ値しか見ない場合、方策は誤差に大きく影響されそうです。
2. 一方で、KL正則化がある場合は過去のQ値も参考にするため、誤差の影響は小さく抑えられそうですね。しかし、$k$が大きくなるほど更新量が減り、収束が遅くなりそうなこともわかります。
3. KL＋エントロピーはこの中間を取っているとみなせそうです。過去の全ての情報を覚えず、古い情報を忘れることで、更新量をなるべく一定に保つことができます。つまり、「誤差に対する頑健性」と「収束速度」を$\beta$が調整しています。

---

### 誤差伝搬解析


正則化が入った方策反復法の誤差バウンドを次に示します。

$$
\left\|q_*^{\tau}-q_{\pi_{k+1}}^{\tau}\right\|_{\infty} \leq \frac{2}{(1-\gamma)^2}\left((1-\gamma) \sum_{j=1}^k \gamma^{k-j}\left\|E_j^\beta\right\|_{\infty}\right)+\gamma^k\left(1+\frac{1-\beta}{1-\gamma}\right) \sum_{j=0}^k\left(\frac{\beta}{\gamma}\right)^j v_{\max }^\tau
$$

これを示すために、次のエントロピー版の**価値差分方程式**と**指数的改善定理**が必要になります。

---

**補題１：価値差分方程式（正則化あり）**

任意の$q$について、

$$
\begin{aligned}
q_{\pi^\prime\mid\mu}^{\lambda, \tau}-q &=T_{\pi^\prime\mid \mu}^{\lambda, \tau} q_{\pi^\prime\mid \mu}^{\lambda, \tau}-T_{\pi^\prime\mid \mu}^{\lambda, \tau} q +T_{\pi^\prime\mid\mu}^{\lambda, \tau} q -q\\
&=\gamma P_{\pi^\prime}\left(q_{\pi^\prime\mid\mu}^{\lambda, \tau}-q\right)+T_{\pi^\prime\mid\mu}^{\lambda, \tau} q-q \\
&\Leftrightarrow q_{\pi^\prime\mid\mu}^{\lambda, \tau}-q=\left(I-\gamma P_{\pi^\prime}\right)^{-1}\left(T_{\pi^\prime\mid\mu}^{\lambda, \tau} q-q\right)
\end{aligned}
$$
なので、

$$
q_{\pi^\prime\mid\mu}^{\lambda, \tau}-q=\left(I-\gamma P_{\pi^\prime}\right)^{-1}\left(T_{\pi^\prime\mid\mu}^{\lambda, \tau} q-q\right)
$$

が成立します。

---

この価値差分方程式を使うと、次のように目的関数を変形することができます。

$$
\begin{aligned}
q_*^\tau-q_{\pi_{k+1}}^\tau & =q_*^\tau-h_k+h_k-q_{\pi_{k+1}}^\tau \\
& =\left(I-\gamma P_{\pi_{*}}\right)^{-1}\left(T_{\pi_{*}}^{0, \tau} h_k-h_k\right)-\left(I-\gamma P_{\pi_{k+1}}\right)^{-1}\left(T_{\pi_{k+1}}^{0, \tau} h_k-h_k\right) .
\end{aligned}
$$

よって、$T_{\pi_{k+1}}^{0, \tau} h_k-h_k$をバウンドすれば良いことがわかります。

TODO: こっから先の導出いまいちわからないな... 多分わかりやすい導出があるんだけど思いついていない... Value Iterationのときならいけるが、方策反復やλ方策反復になると$T_{\pi_{k+1}}^{0, \tau} h_k-h_k$のバウンドが出せそうにない... ギブアップ

## $\lambda$-方策反復法

**参考**
* [Performance bound for Approximate Optimistic Policy Iteration](https://hal.inria.fr/inria-00480952/document)
* [Leverage the Average: an Analysis of KL Regularization in Reinforcement Learning](https://arxiv.org/abs/2003.14089)：表記を参考にしました

方策反復法では$(T_{\pi_{k+1}})^\infty$を使っていました。
一方で、Q学習や[マルチステップ方策評価](RL_multi_step.ipynb)のように、様々な形式の方策評価が存在します。
そこで、次のような**一般的な**近似方策反復法を考えます。

$$
\begin{cases}
\pi_{k+1} \in \mathcal{G}(q_k) \\
q_{k+1} = \sum_{n \geq 1} \lambda_n (T_{\pi_{k+1}})^n q_k + \epsilon_{k+1}
\end{cases}
$$

ここで、$\sum_{n>1} \lambda_n=1$はマルチステップ方策評価の重み係数です。詳しくは[マルチステップ方策評価](RL_multi_step.ipynb)を参考にしてください。
この誤差解析をしてみましょう。

### 誤差伝搬解析

---

**直感的な方針**

まず、$q_* - q_{\pi_k}=q_* - q_k + q_k - q_{\pi_k}$のように分解します。
そして「$q_k$, $q_{\pi_k}$, $q_*$はなんとなく近いだろう」という方針のもとで、$q_k - q_{\pi_k}$や$q_k - q_*$をバウンドしていきます。

---

まず、
$$
\begin{aligned}\left\|q_*-q_{\pi_k}\right\|_{\infty} & =\max \left(q_*-q_{\pi_k}\right) \\ & =\max \left(q_*-q_k+\epsilon_k+q_k-\epsilon_k-q_{\pi_k}\right) \\ & =\max \left(d_k+s_k\right) \\ & \leq \max d_k+\max s_k\end{aligned}
$$